# Relevance Logic Examples

This notebook demonstrates relevance reasoning using the Logos theory framework. We explore the **relevance** operator (≼) and its relationship with grounding, essence, and identity, showing both invalid principles through countermodels and valid theorems.

## Overview

Relevance logic extends constitutive logic with a relevance operator that captures when one proposition is relevant to another. Unlike classical implication, relevance requires a genuine connection between propositions:
- **Relevance** (≼): When one proposition is relevant to another
- Relates to **grounding** (≤) and **essence** (⊑) through specific principles
- Provides foundations for relevant implication and avoiding paradoxes of material implication

## Contents
1. **Setup** - Load the relevance theory and operators
2. **Antecedent Strengthening Countermodel** - Relevance fails strengthening
3. **Relevance to Ground Theorem** - Relevance with conjunction yields grounding
4. **Strict Implication vs Relevance Countermodel** - Necessary implication doesn't ensure relevance
5. **Grounding Implies Relevance Theorem** - Grounding ensures relevance

In [ ]:
import sys
from model_checker.jupyter import create_build_example
from model_checker.theory_lib.logos.semantic import LogosSemantics, LogosProposition, LogosModelStructure
from model_checker.theory_lib.logos.operators import LogosOperatorRegistry

# Create operator registry for relevance theory
# We need constitutive and modal operators as well
relevance_registry = LogosOperatorRegistry()
relevance_registry.load_subtheories(['extensional', 'modal', 'constitutive'])

# Build the semantic theory dictionary
relevance_theory = {
    "semantics": LogosSemantics,
    "proposition": LogosProposition,
    "model": LogosModelStructure,
    "operators": relevance_registry.get_operators(),
}

print("=" * 70)
print("RELEVANCE THEORY LOADED")
print("=" * 70)
print("\nAvailable operators:")
print("  - Relevance: \\preceq (≼)")
print("  - Ground: \\leq (≤)")
print("  - Essence: \\sqsubseteq (⊑)")
print("  - Identity: \\equiv (≡)")
print("\nThe theory also includes classical and modal operators for complex formulas.")
print("=" * 70)

## Example 1: Antecedent Strengthening Countermodel

### Background
Like other hyperintensional operators, relevance fails **antecedent strengthening**. Just because A is relevant to B doesn't mean that a stronger proposition (A ∧ C) is also relevant to B. Relevance is sensitive to the specific content of propositions.

### The Argument
We test whether conjunction on the left preserves relevance:
- **Invalid Conclusion**: (A ∧ B) ≼ A

This should fail because the conjunction may introduce irrelevant content.

In [ ]:
# REL_CM_1: Test antecedent strengthening
REL_CM_1_example = [
    [],                                    # No premises
    ['((A \\wedge B) \\preceq A)'],        # Conclusion
    {
        'N': 4,
        'contingent': True,
        'non_null': True,
        'non_empty': True,
        'disjoint': False,
        'max_time': 10,
        'iterate': 2,
        'expectation': True,  # We expect to find a countermodel
    }
]

print("Running model checker...")
model = create_build_example('REL_CM_1', relevance_theory, REL_CM_1_example)

# Display the countermodel if found
if model.model_structure.z3_model:
    model.model_structure.print_to(
        model.settings,
        'REL_CM_1',
        'Antecedent Strengthening',
        output=sys.stdout
    )
else:
    print("No countermodel found (unexpected - strengthening should fail)")

### Result Interpretation

The countermodel shows that adding conjuncts can destroy relevance. While A alone might be relevant to something, conjoining it with B introduces additional content that may not preserve the relevance relationship. This is crucial for avoiding irrelevant premises in logical arguments.

---

## Example 2: Relevance to Ground Theorem

### Background
There is a deep connection between relevance and grounding through conjunction. If A is relevant to B, then the conjunction (A ∧ B) grounds B. This captures the idea that relevant propositions, when combined with their target, provide grounding.

### The Argument
We test whether the following inference is valid:
- **Premise**: A ≼ B (A is relevant to B)
- **Conclusion**: (A ∧ B) ≤ B (A and B together ground B)

This reveals how relevance contributes to grounding relationships.

In [ ]:
# REL_TH_1: Test relevance to conjunction/ground
REL_TH_1_example = [
    ['(A \\preceq B)'],                   # Premise
    ['((A \\wedge B) \\leq B)'],           # Conclusion
    {
        'N': 3,
        'contingent': False,
        'non_null': True,
        'non_empty': True,
        'disjoint': False,
        'max_time': 10,
        'iterate': 1,
        'expectation': False,  # We expect NO countermodel (theorem is valid)
    }
]

print("Running model checker...")
model = create_build_example('REL_TH_1', relevance_theory, REL_TH_1_example)

# Display result
if model.model_structure.z3_model:
    print("UNEXPECTED: Found a countermodel to a theorem!")
    model.model_structure.print_to(
        model.settings,
        'REL_TH_1',
        'Relevance to Ground',
        output=sys.stdout
    )
else:
    print("=" * 70)
    print("THEOREM VALIDATED: Relevance to Ground")
    print("=" * 70)
    print("No countermodel found - the inference is VALID")
    print("\nThis confirms that: (A ≼ B) ⊨ ((A ∧ B) ≤ B)")
    print("If A is relevant to B, then A and B together ground B.")
    print("=" * 70)

### Result Interpretation

This theorem establishes a fundamental connection between relevance and grounding. When A is relevant to B, conjoining them creates a grounding relationship. This shows how relevance participates in constitutive explanation.

---

## Example 3: Strict Implication vs Relevance Countermodel

### Background
Necessary implication (strict implication) does not guarantee relevance. Just because A necessarily implies B doesn't mean A is relevant to B. Classic examples include necessary truths that imply everything but aren't relevant to most things.

### The Argument
We test whether the following inference is valid:
- **Premise**: □(A → B) (necessarily, if A then B)
- **Invalid Conclusion**: A ≼ B (A is relevant to B)

This should fail because necessary connection doesn't establish relevance.

In [ ]:
# REL_CM_7: Test strict implication to relevance
REL_CM_7_example = [
    ['\\Box (A \\rightarrow B)'],         # Premise
    ['(A \\preceq B)'],                   # Conclusion
    {
        'N': 3,
        'contingent': True,
        'non_null': True,
        'non_empty': True,
        'disjoint': False,
        'max_time': 10,
        'iterate': 1,
        'expectation': True,  # We expect to find a countermodel
    }
]

print("Running model checker...")
model = create_build_example('REL_CM_7', relevance_theory, REL_CM_7_example)

# Display the countermodel if found
if model.model_structure.z3_model:
    model.model_structure.print_to(
        model.settings,
        'REL_CM_7',
        'Strict Implication to Relevance',
        output=sys.stdout
    )
else:
    print("No countermodel found (unexpected - strict implication should not entail relevance)")

### Result Interpretation

The countermodel demonstrates that necessary implication doesn't establish relevance. This is crucial for relevance logic's solution to the paradoxes of material and strict implication - just because something is necessarily true doesn't make it relevant to arbitrary propositions.

---

## Example 4: Grounding Implies Relevance Theorem

### Background
While strict implication doesn't ensure relevance, grounding does. If A grounds B, then A is relevant to B. This shows that grounding is a stronger relation that encompasses relevance.

### The Argument
We test whether the following inference is valid:
- **Premise**: A ≤ B (A grounds B)
- **Conclusion**: A ≼ B (A is relevant to B)

This reveals the hierarchy of hyperintensional relations.

In [ ]:
# REL_TH_7: Test grounding to relevance
REL_TH_7_example = [
    ['(A \\leq B)'],                      # Premise
    ['(A \\preceq B)'],                   # Conclusion
    {
        'N': 3,
        'contingent': False,
        'non_null': True,
        'non_empty': True,
        'disjoint': False,
        'max_time': 10,
        'iterate': 1,
        'expectation': False,  # We expect NO countermodel (theorem is valid)
    }
]

print("Running model checker...")
model = create_build_example('REL_TH_7', relevance_theory, REL_TH_7_example)

# Display result
if model.model_structure.z3_model:
    print("UNEXPECTED: Found a countermodel to a theorem!")
    model.model_structure.print_to(
        model.settings,
        'REL_TH_7',
        'Grounding to Relevance',
        output=sys.stdout
    )
else:
    print("=" * 70)
    print("THEOREM VALIDATED: Grounding Implies Relevance")
    print("=" * 70)
    print("No countermodel found - the inference is VALID")
    print("\nThis confirms that: (A ≤ B) ⊨ (A ≼ B)")
    print("If A grounds B, then A is relevant to B.")
    print("\nThis shows grounding is stronger than relevance.")
    print("=" * 70)

### Result Interpretation

This theorem establishes that grounding implies relevance. The hierarchy shows:
- Grounding is the strongest relation (implies relevance)
- Relevance is weaker (doesn't imply grounding)
- Strict implication is orthogonal (neither implies nor is implied by relevance)

---

## Summary

These examples demonstrate key properties of relevance logic:

### Countermodels (Invalid Principles)
- **Antecedent Strengthening Fails**: Adding conjuncts can destroy relevance
- **Strict Implication vs Relevance**: Necessary connection doesn't ensure relevance

### Theorems (Valid Principles)  
- **Relevance to Ground**: Relevance with conjunction yields grounding
- **Grounding Implies Relevance**: Grounding is stronger than relevance

These properties make relevance logic suitable for:
- Avoiding paradoxes of material and strict implication
- Ensuring genuine connections between premises and conclusions
- Analyzing relevant entailment and conditional reasoning
- Building foundations for relevant logics and information flow